Problem 2

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, t
from statsmodels.tsa.arima.model import ARIMA
from scipy.optimize import minimize
import statsmodels.api as sm

In [2]:
# Implement return_calculate()
def return_calculate(prices: pd.DataFrame, method: str = "DISCRETE", dateColumn: str = "date") -> pd.DataFrame:
    vars = list(prices.columns)
    nVars = len(vars)
    vars.remove(dateColumn)
    vars = list(map(str, vars))
    if nVars == len(vars):
        raise ValueError("dateColumn: " + dateColumn + " not in DataFrame: " + str(vars))
    nVars -= 1

    p = prices[vars].values
    n, m = p.shape
    p2 = np.empty((n-1, m), dtype=np.float64)

    for i in range(n-1):
        for j in range(m):
            p2[i,j] = p[i+1,j] / p[i,j]

    if method.upper() == "DISCRETE":
        p2 -= 1.0
    elif method.upper() == "LOG":
        p2 = np.log(p2)
    else:
        raise ValueError("method: " + method + " must be in (\"LOG\",\"DISCRETE\")")

    dates = prices[dateColumn].iloc[1:].reset_index(drop=True)
    out = pd.DataFrame({dateColumn: dates})
    for i in range(nVars):
        out[vars[i]] = p2[:,i]
    return out

In [4]:
# Import data
data = pd.read_csv("DailyPrices.csv", index_col=0)

In [5]:
# Calculate arithmetic returns
returns = data.pct_change().dropna()

In [8]:
# Break the returns for META into 2 groups, a modeling group and a holdout sample
meta_returns = returns["META"]
meta_modeling_returns = meta_returns[:-60]
meta_holdout_returns = meta_returns[-60:]

In [10]:
# Remove mean from META series
meta_modeling_returns -= meta_modeling_returns.mean()

In [11]:
# Calculate VaR using normal distribution
alpha = 0.05
n = 10000
sigma = meta_modeling_returns.std()
r = np.random.normal(0, sigma, n)
VaR = -np.quantile(r, alpha)

print("VaR for normal distribution is",VaR)

VaR for normal distribution is 0.06616401296978026


In [12]:
# Calculate VaR using normal distribution with an Exponentially Weighted variance (lambda = 0.94)
l = 0.94
tw = 0
m = len(meta_modeling_returns)
w = np.zeros(m)
for i in range(m):
    w[i] = (1 - l) * l ** (m - i - 1)
    tw += w[i]
for i in range(m):
    w[i] = w[i] / tw

sigma_ew = np.sqrt((meta_modeling_returns * w).T @ meta_modeling_returns)
r_ew = np.random.normal(0, sigma_ew, n)
VaR_ew = -np.quantile(r_ew, alpha)

print("VaR for normal distribution with exponentially weighted variance is:", VaR_ew)

VaR for normal distribution with exponentially weighted variance is: 0.0970295881134886


In [13]:
# Calculate VaR using MLE fitted T distribution
returns = meta_modeling_returns
negLL = lambda params, returns: -1 * np.sum(t.logpdf(returns, df=params[0], loc=params[1], scale=params[2]))

constraints = ({"type": "ineq", "fun": lambda x: x[0] - 1},
               {"type": "ineq", "fun": lambda x: x[2]})

returns_t = minimize(negLL, x0=[10, np.mean(returns), np.std(returns)], args=returns, constraints=constraints)
df, loc, scale = returns_t.x[0], returns_t.x[1], returns_t.x[2]
sim_returns = t.rvs(df, loc=loc, scale=scale, size=n)
VaR_t = -np.percentile(sim_returns, alpha*100)

print("VaR for MLE fitted T Distribution is:", VaR_t)

VaR for MLE fitted T Distribution is: 0.05949624391371723


In [18]:
# Calculate VaR using fitted AR(1) model
ar1_fit = sm.tsa.arima.ARIMA(meta_modeling_returns, order=(1, 0, 0))
con,beta,s= ar1_fit.fit().params[0],ar1_fit.fit().params[1],np.sqrt(ar1_fit.fit().params[2])
esim = np.random.normal(0,s,n)
r = np.zeros(n)
for i in range(n):
    r[i]=con+meta_modeling_returns.iloc[-1]*beta+esim[i]
VaR = -np.quantile(r,alpha)

print("VAR for fitted AR(1) is: {}".format(VaR))

c:\program files\python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\program files\python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\program files\python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


VAR for fitted AR(1) is: 0.06500907772777474


In [20]:
# Calculate VaR using historic simulation
rsim = np.random.choice(meta_modeling_returns,n)
VaR = -np.quantile(rsim,alpha)

print("VAR for Historic Simulation is: {}".format(VaR))

VAR for Historic Simulation is: 0.05895278422859425
